In [1]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras, feature_column
from sklearn import model_selection, preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_hub as hub
import PIL.Image as Image
import tensorflow_datasets as tfds

from IPython.core.interactiveshell import InteractiveShell


In [2]:
# 配置项
# 这个要放到设置中文之前否则还是小方框
plt.style.use("seaborn")

# 指定默认字体 用来正常显示中文标签
plt.rcParams['font.sans-serif'] = ['SimHei']
# 解决保存图像是负号'-'显示为方块的问题
plt.rcParams['axes.unicode_minus'] = False

# #全部行都能输出
InteractiveShell.ast_node_interactivity = "all"

In [5]:
# 自定义layer层 需要自己实现下面三个方法  
class MyLayer(keras.layers.Layer):
    def __init__(self, num_outputs):
        super(MyLayer, self).__init__()
        # 下面就可以执行你自己从操作
        self.num_outputs = num_outputs
    
    def build(self, input_shape):
        print(input_shape, input_shape[-1])
        self.kernel = self.add_variable('kernel', shape=[int(input_shape[-1]), self.num_outputs])
        # 下面可以自己想要的操作
    def call(self, input):
        return tf.matmul(input, self.kernel)
    
# 
my_layer = MyLayer(10)
print(my_layer(tf.zeros([10, 5])))
print(my_layer.trainable_variables)

(10, 5) 5
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(10, 10), dtype=float32)
[<tf.Variable 'my_layer_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[-0.5222734 , -0.35850587,  0.30134052,  0.29253757, -0.53505194,
        -0.53600067,  0.36402172,  0.23172945, -0.2801789 , -0.30622697],
       [ 0.14803815,  0.2647987 ,  0.33990455,  0.41787618,  0.253309  ,
         0.02150285, -0.19001603,  0.2552945 ,  0.11901969, -0.5368032 ],
       [ 0.24577576, -0.14686516, -0.19569522,  0.11464286,  0.45634454,
         0.04580432,  0.4151029 , -0.17807671,  0.07223165,  0.6227421 ],
       [ 0.06573921,  0.39409167, -0.07556933,  0.3521979 , -0.4884317 ,
         0.5813101 , -0.203162

In [7]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: id=466, shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [11]:
# 自定义模型 模型是有很多layer组成的  如上所示
class MyModel(keras.Model):
    def __init__(self, kernel_size, filters):
        super(MyModel, self).__init__(name='jqc')
        print('-------========-----')
        print(filters)
        print('-----******-------')
        filters1, filters2, filters3 = filters
        
        self.conv2a = keras.layers.Conv2D(filters1, (1, 1))
        self.bn2a = keras.layers.BatchNormalization()
        
        self.conv2b = keras.layers.Conv2D(filters2, kernel_size, padding='same')
        self.bn2b = keras.layers.BatchNormalization()
        
        self.conv2c = keras.layers.Conv2D(filters3, (1, 1))
        self.bn2c = keras.layers.BatchNormalization()
    
    def call(self, input_tensor, training=False):
        
        x = self.conv2a(input_tensor)
        x = self.bn2a(x, training=training)
        x = tf.nn.relu(x)
        
        # 前一项的输出当做后一项的输入
        x = self.conv2b(x)
        x = self.bn2b(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2c(x)
        x = self.bn2c(x, training=training)

        x += input_tensor
        return tf.nn.relu(x)


In [12]:
model = MyModel(1, [1, 2, 3])
print(model(tf.zeros([1, 2, 3, 3])))
print([x.name for x in model.trainable_variables])

-------========-----
[1, 2, 3]
-----******-------
tf.Tensor(
[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]], shape=(1, 2, 3, 3), dtype=float32)
['jqc/conv2d_6/kernel:0', 'jqc/conv2d_6/bias:0', 'jqc/batch_normalization_6/gamma:0', 'jqc/batch_normalization_6/beta:0', 'jqc/conv2d_7/kernel:0', 'jqc/conv2d_7/bias:0', 'jqc/batch_normalization_7/gamma:0', 'jqc/batch_normalization_7/beta:0', 'jqc/conv2d_8/kernel:0', 'jqc/conv2d_8/bias:0', 'jqc/batch_normalization_8/gamma:0', 'jqc/batch_normalization_8/beta:0']
